<a href="https://colab.research.google.com/github/CCS-ZCU/pribehy-dat/blob/master/scripts/7_tei-xml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Digitální edice ve standardu TEI-XML

autor: Vojtěch Kaše (kase@kfi.zcu.cz)

V této kapitole se budeme věnovat práci s digitálními editacemi vytvořenými podle standardu [TEI-XML](https://tei-c.org). TEI-XML je způsob jak digitalizované literární dílo veřejně prezentovat tak, aby z něj zejména badatelé a studenti z oblasti humanitních věd mohli získávat hodnotné informace například skrz pokročilé prohlédávání. Za tímto účelem jsou texty v těchto edicích ve větší či menší míře buď ručně nebo polautomaticky anotované. Tím se myslí, že jsou v těchto textech označeny specifické výrazy, jako jsou názvy osob či organizací, místní názvy nebo jiné důležité termíny. Stejně tak jsou v záhlaví těchto dokumentů standardizovaným způsobem uvedena nejrůznější metadata, typicky data o autorovi daného textu, jeho vydavateli apod. Projektů, které dnes využívají tento standard, je obrovské množství. Rozsáhlý výčet lze nalézt [zde](https://tei-c.org/Activities/Projects/). 

Důkladné obeznámení se s těmito standardy by vydalo na celý samostatný kurz. Zde však budeme postupovat obdobně jako v jiných kapitolách a pouze se uvnitř prostředí Jupyter notebooku prohlédneme, jak některé takovéto digitalizované edice vypadají a jak s nimi můžeme pracovat.

TEI-XML je spojení dvou akronymů: TEI = Text Encoding Initiative a XML = Extensible Markup Language.
 
**XML** je definováno jako *rozšiřitelný značkovací jazyk* a stejnojmenný formát souborů využívaný zejména pro výměnu dat mezi aplikacemi a pro publikování dokumentů (viz [wikipedia](https://cs.wikipedia.org/wiki/Extensible_Markup_Language). V mnoha ohledech se běžné XML dokumenty podobají kódu HTML stránek (jak uvidíme néže, pro práci s nimi se v rámci Pythonu i využívají stejné nástroje). Na rozdíl od HTML stránek však data naformátovaná za využití XML standardu nejsou určena pro webové prohlížeče.

XML soubor sestává z prvků (elementů), které jsou vyznačeny *tagy*, neboli značkami. Tyto tagy jsou vyznačeny pomocí symbolů `<` a `>`, jsou definovány tvůrcem a tvůrce si může vytvářet zcela vlastní tagy. Vždy zde však musí být právě jeden kořenový prvek, tzv. root. V XML je kořenový prvek jedinečný a nemá svůj vlastní obsah, ale pouze obaluje všechny další prvky v dokumentu. 

Tagy mohou a často obsahují atributy, které definují dodatečné vlastnosti příslušných prvků nebo data provazují s dalšími daty, často pomocí hypertextových odkazů. Na rozdíl od HTML musí být v rámci XML všechny tagy tzv. párové. Tag v podobě `<element1>` prvek otvírá a v podobě `</element>` jej uzavírá. Vše mezi tím je obsahem daného prvku. Prvky se do sebe často vnořují, čímž dokument získává určitou stromovou a navigovatelnou strukturu. 

Nejlépe bude struktura xml souboru vidět na následujícím minimalistickém příkladu:

```xml
<?xml version="1.0" encoding="UTF-8"?>
<root>
    <element1 attribute1="value1">Text for element1</element1>
    <element2 attribute2="value2">Text for element2</element2>
</root>
```
Inciativa **TEI** přichází se standardy pro to, jak tento formát dat použít pro zveřejňování dokumentů, jako jsou literární díla, básně, rozhovory, dopisy, nápisy, papyry a mnoho dalšího (existuje např. i verze pro obrazová díla či hudební notaci). TEI byla ustavena v roce 1987 s následujícím cílem "vyvíjet, udžovat a propagovat metody pro kódování humanitních dat v elektronické podobě
 nezávislé na konkrétním hardweru a softwaru ("to develop, maintain, and promulgate hardware- and software-independent methods for encoding humanities data in electronic form" - viz [zde](https://tei-c.org/about/history/)).
 
Veškerý text daného dokumentu se nachází uvnitř prvku `<body></body>`. Ten se může standardizovaným způsoben dále dělit na kapitoly, verše či řádky, či průpovědi jednotlivých mluvčích, v závislosti na žánru daného dokumentu (všechny tyto možnosti jsou detailně popsány v Guidlines [zde](https://tei-c.org/release/doc/tei-p5-doc/en/html/index.html)). Stejně tak jsou jasně nadefinovány způsoby, jak v rámci textu vyznačit výskyty jmenných entit, jako jsou jména osob či místní názvy, nebo například datumy (viz [zde](https://tei-c.org/release/doc/tei-p5-doc/en/html/ND.html). Tyto stanardy si ukážeme na konkrétníh příkladech níže. Pro detailnější obeznámení pak doporučiji například online kurz [zde](https://teach.dariah.eu/course/view.php?id=32). Praktický úvodní text v češtině je dostupný např. [zde](https://digital-humanities.phil.muni.cz/clanky/metadata-co-ma-spolecneho-netflix-a-digitalni-humanitni-vedy).

Jak již bylo uvedeno výše, XML dokumenty mají v mnohém obdobné vlastnosti jako webové stránky ve formátu HTML a k práce s nimi lze použít podobné nástroje. Ač TEI-XML dokumenty nejsou určeny pro webové prohlížeče, lze je často nalézt přímo na webu, např. Nezříka jsou také zpřístupňovány pomocí API. V následujícím civčení proto budeme používat v podstatě stejné nástroje jako v případě kapitol o HTTP a websrapingu a využívání API, zejména knihovny `requests` a `BeautifulSoup`.

In [7]:
import requests
from bs4 import BeautifulSoup

### Korpus německé poezie

Jako první příklad vybereme báseň z korpusu německé poezie **Deutsches Lyrik Korpus** (DLK), dostupného na platformě GitHub [zde](https://github.com/tnhaider/DLK). Jednotlivé básně naformátované ve formě TEI-XML souborů se nachází v této podsložce čítající více než 60,000 položek: https://github.com/tnhaider/DLK/tree/master/DLK/tei/tei_plain.

Vybereme náhodný soubor, otevřeme jej, a zobrazíme jej v "raw" podobě (v oknu souboru klikneme v pravém horním rohu). URL adresu souboru v syrové podobě si zkopírujeme a vložíme jako hodnotu proměnné `url` v buňce
 níže. (Tato url adresa se skládá z informace u umístění souboru v rámci Githubu, která je pro všechny soubory shodná a samotného názvu souboru.)




In [16]:
url = "https://raw.githubusercontent.com/tnhaider/DLK/master/DLK/tei/tei_plain/dta.poem.10006-Zachariae%2C_Justus_Friedrich_Wilhelm-Die_Vergnügungen_der_Melancholey..tei.xml"
resp = requests.get(url)
resp

<Response [200]>

Načtený soubor (v proměnné `resp`) následně parsujeme pomocí knihovny `BeuatifulSoup` do proměnné `soup`, pomocí čehož získáme přístup k stromové struktuře daných dat. Obsah si vypíšeme.

In [17]:
soup = BeautifulSoup(resp.text)
soup

<html><body><tei xmlns="http://www.tei-c.org/ns/1.0">
<teiheader>
<filedesc>
<titlestmt>
<title type="main">Die Vergnügungen  
  der Melancholey.</title>
<author>
<persname>
<surname>Zachariae</surname>
<forename>Justus Friedrich Wilhelm</forename>
</persname>
</author>
<editor>
<persname>
<surname>Haider</surname>
<forename>Thomas Nikolaus</forename>
</persname>
<orgname>University of Göttingen/Max Planck Institute for Empirical Aesthetics, Frankfurt am Main</orgname>
<email>mail@thomashaider.net</email>
</editor>
</titlestmt>
<editionstmt>
<edition>
<name>Deutsches Lyrik Korpus Edition (DLK) v6</name>
<date>9-11-2022</date>
</edition>
</editionstmt>
<extent>
<measure type="stanzas">1</measure>
<measure type="verses">22</measure>
<measure type="verses_per_stanza">1-22</measure>
<measure type="syllables">290</measure>
<measure type="tokens">179</measure>
<measure type="characters">1101</measure>
</extent>
<publicationstmt>
<publisher>
<name></name>
</publisher>
<pubplace></pubplace>
<d

Vidíme kód s jednotlivými prvky. V samotném úvodu vidíme prvky definující metadata o dané básni: jméno autora uvnitř prvku `<persname>`, do sebe vnořuje ještě dva podprvky `<surname>` a `<lastname>`. Všechna tato metadata se nacházejí uvnitř prvku `<teiheader>`. Po jeho uzavření následuje prvek `<text>`, v níž se nachází text příslušné básně. 

Nejprve z textu vyzískejme základní metadata. Stačí nám pohybovat se po stromové struktuře stejně, jako jsme to činili v kapitole o webscrapingu. Například k názvu literárního díla se dostaneme takto:

In [19]:
soup.teiheader.title

<title type="main">Die Vergnügungen  
  der Melancholey.</title>

V tomto případě však vidíme i tagy se všemi jejich atributy. Chceme-li samotný textový obsah, musíme použít `text`. Hodnotu si uložíme do proměnné a vypíšeme si ji.




In [25]:
title = soup.teiheader.title.text
print(title)

Die Vergnügungen  
  der Melancholey.


Stejně můžeme postupovat v případě dalších prvků:

In [30]:
author_surname = soup.teiheader.author.surname.text
author_forename = soup.teiheader.author.forename.text

Vypišme si nyní celé jméno autora a příslušné básně.

In [37]:
print(author_forename, author_surname, "-", title)

Justus Friedrich Wilhelm Zachariae - Die Vergnügungen  
  der Melancholey.


Pozor, že některé tagy se zde mohou objevovat opakovaně. Pomocí metody výše jsme si vypsali pouze první výskyt daného tagu. Chceme-li vypsat všechny výskyty, musíme na objekt `soup` použít metodu `findall()`. Vyzkoušíme s prvkem `surname`.

In [39]:
soup.find_all("surname")

[<surname>Zachariae</surname>, <surname>Haider</surname>]

V případě některých básní budeme mít jmen víc. Je to proto, že je zde ještě příjmení editora dané publikace. Proto jsme výše byli konkrétnější a specifikovali, že se v rámci objektu `soup` pohybujeme v prvku autor.

Nyní se přesuňme k textu samotné básně. Ta je v rámci objektu `soup` dostupná v prvku `text`. V nejsyrovější podobě se k textu básně dostaneme takto.

In [45]:
soup.find("text").get_text() # zde je název tagu "<text>" poněkud matoucí, neboť BeautifulSoup pracuje se stejnojmenným atributem

'\n\n\n          Die Vergnügungen  \n  der Melancholey.\n          \nAndre mögen am lächelnden Abend des Sommers\nWenn sie am dumpfen Geräusch des murmelnden Ba-\nOder das sanftere Roth des streifichten Westens be-\nMich ergetzt nur Nebel und Dunkel des blassen De-\nWenn die Schatten sich dann des langen Abends ge-\nUnd ein schimmernder Stral der matten sterbenden\nDurch den dämmernden Raum, sich bricht: dann laß\nVon dem Jauchzen des Unsinns, das ietzo mit festlichem\nDurch die erleuchteten Zimmer ertönt, dann laß mich\nSitzen, allein nur vergnügt an der niedern klagenden\nSchlummer erweckendem Lied; und laß mich mit mei-\nIn mich gekehrt, den Wechsel der Dinge, die leeren\nUnd die vergebliche Mühe betrachten, die unsrer Er-\nForschen vereitelt, so wie wir die Wüste des Lebens\nDiese gesegnete Stunde der Stille wird alles das Lä-\nSchimmernder Thorheit entdecken, das, gleich des li-\nFalscher zaubrischer Kunst, die allzusicheren Augen\nMit der verborgnen Verblendung getäuscht; den bez

V rámci digitální edice je však báseň dostupná po jednotlivých verších. Pro jednotlivé verše se používá tag `<l>`. Vytvořme si uspořádaný seznam všech veršů:

In [47]:
verses = [l.text for l in soup.find("text").find_all("l")]
verses

['Andre mögen am lächelnden Abend des Sommers',
 'Wenn sie am dumpfen Geräusch des murmelnden Ba-',
 'Oder das sanftere Roth des streifichten Westens be-',
 'Mich ergetzt nur Nebel und Dunkel des blassen De-',
 'Wenn die Schatten sich dann des langen Abends ge-',
 'Und ein schimmernder Stral der matten sterbenden',
 'Durch den dämmernden Raum, sich bricht: dann laß',
 'Von dem Jauchzen des Unsinns, das ietzo mit festlichem',
 'Durch die erleuchteten Zimmer ertönt, dann laß mich',
 'Sitzen, allein nur vergnügt an der niedern klagenden',
 'Schlummer erweckendem Lied; und laß mich mit mei-',
 'In mich gekehrt, den Wechsel der Dinge, die leeren',
 'Und die vergebliche Mühe betrachten, die unsrer Er-',
 'Forschen vereitelt, so wie wir die Wüste des Lebens',
 'Diese gesegnete Stunde der Stille wird alles das Lä-',
 'Schimmernder Thorheit entdecken, das, gleich des li-',
 'Falscher zaubrischer Kunst, die allzusicheren Augen',
 'Mit der verborgnen Verblendung getäuscht; den bezau-',
 'Uns zu t

### Dokumenty EHRI

Jako další příklad použijeme digitální edice vydané projektem [EHRI](https://www.ehri-project.eu) (=The European Holocaust Research Infrastructure). EHRI sestává z mnoha podprojektů. Jedním z nich jsou přepisy raných svědectví přeživších holocaustu ([Early Holocaust Testimony](https://early-testimony.ehri-project.eu)). Tyto edice jsou velice důkladně označkovány ohledně vystujících se jmen osob, místních názvů i organizací, jimž jsou přiřazeny jednoznačné identifikátory, které jsou spravovány v samostatných databázích (vocabularies). Jedná se o vzorový příklad tzv. [Linked Open Data](https://www.ontotext.com/knowledgehub/fundamentals/linked-data-linked-open-data/).    

Vyjdeme  z rozcestníku jednotlivých svědectví dostupného [zde](https://early-testimony.ehri-project.eu/search). Digitální edice ve formátu XML je u každého svědectví dostupná ke stažení přes tlačítko v pravém horním rohu. My však nemusíme soubor fyzicky stahovat, pouze si zkopírujeme URL adresu, na kterou tlačítko pro stažení XML směřuje. 

In [48]:
url = "https://ehri-editions-uh.s3.eu-central-1.amazonaws.com/original/e90886e26c700a8966e2355f023df678.xml"
resp = requests.get(url)
resp

<Response [200]>

In [50]:
soup = BeautifulSoup(resp.text, features="xml")
soup

<?xml version="1.0" encoding="utf-8"?>
<?xml-model href="http://www.tei-c.org/release/xml/tei/custom/schema/relaxng/tei_all.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"?><!--EHRI ODT to TEI convertor: 2018-01-08T13:51:56+01:00--><TEI xml:id="EHRI-UH-19380803_CS" xmlns="http://www.tei-c.org/ns/1.0">
<teiHeader>
<fileDesc>
<titleStmt>
<title>Povolení Ústředny pro sociální péči Židovské náboženské obce ve Vídni zakoupit
          jízdenku pro jízdu Alexandera Spitze do Prahy</title>
</titleStmt>
<editionStmt>
<edition>Hranice útěku. Rakouští uprchlíci na československých hranicích v roce 1938 </edition>
<author>
<persName>Michal Frankl</persName>
</author>
<author>
<persName>Wolfgang Schellenbacher</persName>
</author>
<sponsor>Zukunftsfonds der Republik Österreich</sponsor>
<sponsor>Grantová agentura ČR</sponsor>
</editionStmt>
<publicationStmt>
<publisher>
<ref target="https://www.ehri-project.eu/">European Holocaust Research
            Infrastructure<

### Britská poezie 19. století

Velice detailně anotovaná data zde: https://www.eighteenthcenturypoetry.org/authors/.